# Выбор локации для скважины

Шаги для выбора локации:

- Построение модели, прогнозирующей объем запасов;
- Выбор месторождения с самым высокими показателями;
- Расчет прибыли равной суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [63]:
#импорт требуемых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [64]:
#сохраняем данные файлов в переменных

In [65]:
#Выодим данные на экран
display(region_1.head(5))
display(region_2.head(5))
display(region_3.head(5))

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [66]:
#Изучем датасеты
region_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [67]:
region_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [68]:
region_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Данные не содержат пропусков в данных, однако присутсвуют строковые значения в переменной id. Данная переменная не несет смысловой нагрузки для исследования и для избежания ошибок при построении моделей удалим ее из каждого датасета.

In [69]:
region_1.drop(["id",],axis=1,inplace=True)
region_2.drop(["id",],axis=1,inplace=True)
region_3.drop(["id",],axis=1,inplace=True)
display(region_1.head(5))
display(region_2.head(5))
display(region_3.head(5))

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


f0,f1,f2 - признаки, мы не знаем их интерпретации поэтому дальнейший отдельный их анализ проводить не будем. Данные в признаках приблизительно равного масштаба, дальнейшее преобразование не требуется.

In [70]:
#проверим взаимосвязь переменных
print(region_1.corr())
print(region_2.corr())
print(region_3.corr())

               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000
               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000
               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000


Вывод: Можно заметить относительно значимую взимосвязь между f2 и product в первом и втором регионе, явную взаимосвязь между f2 и product во втором регионе. В рамках данного иследования данные синтетические и мы не берем данный факт во внимание. Данные подготовлены для формирования моделей.

## Обучение и проверка модели

Для каждого региона предскажем среднее значение запаса сырья с помощью модели линейной регрессии:

In [71]:
target_1 = region_1['product']
features_1 = region_1.drop(['product'] , axis=1)
#Разобъем данные на обучающую и валидационную выборки в соотношении 75:25.
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)#обучим модель
predicted_valid_1 = model_1.predict(features_valid_1)#сделаем предсказания на валидационной выборке и сохраним их
print("Средний запас предсказанного сырья =",predicted_valid_1.mean())
mse = mean_squared_error(target_valid_1, predicted_valid_1)
print("RMSE =", mse ** 0.5)

Средний запас предсказанного сырья = 92.59256778438035
RMSE = 37.5794217150813


In [72]:
target_2 = region_2['product']
features_2 = region_2.drop(['product'] , axis=1)
#Разобъем данные на обучающую и валидационную выборки в соотношении 75:25.
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)#обучим модель
predicted_valid_2 = model_2.predict(features_valid_2)#сделаем предсказания на валидационной выборке и сохраним их
print("Средний запас предсказанного сырья =",predicted_valid_2.mean())
mse = mean_squared_error(target_valid_2, predicted_valid_2)
print("RMSE =", mse ** 0.5)

Средний запас предсказанного сырья = 68.728546895446
RMSE = 0.893099286775617


In [73]:
target_3 = region_3['product']
features_3 = region_3.drop(['product'] , axis=1)
#Разобъем данные на обучающую и валидационную выборки в соотношении 75:25.
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(
    features_3, target_3, test_size=0.25, random_state=12345)

model_3 = LinearRegression()
model_3.fit(features_train_3, target_train_3)#обучим модель
predicted_valid_3 = model_3.predict(features_valid_3)#сделаем предсказания на валидационной выборке и сохраним их
print("Средний запас предсказанного сырья =",predicted_valid_3.mean())
mse = mean_squared_error(target_valid_3, predicted_valid_3)
print("RMSE =", mse ** 0.5)

Средний запас предсказанного сырья = 94.96504596800489
RMSE = 40.02970873393434


Вывод: Большье всего средних предсказанных запасов в третьем регионе, а более точная модель в первом регионе.

## Подготовка к расчёту прибыли

Для удобства дальнейших расчетов сохраним все значимые параметры в переменных, а также расчитаем значение безубыточного объема сырья для скважины.

In [74]:
#сохраним ключевые знвчения в отдельных переменных
point_income = 450000
investments = 10000000000
points = 500
best_points = 200

In [75]:
#рассчитаем достаточный объём сырья для безубыточной разработки новой скважины
volume = (investments/point_income)/best_points
print(round(volume))

111


Вывод: для того, чтобы при разработке месторождения выйти хотя бы в ноль, необходимо, чтобы средний запас скважин был не менее 111. Запас сырья в моделях запущенных ранее ниже данного значения.

## Расчёт прибыли и рисков 

Создадим функцию расчитывающую прибыль по скважинам с максимальными значениями предсказаний (на основе 200 лучших):

In [76]:
#Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели
def revenue(target, predictions):
    predictions = pd.Series(predictions)
    top_pred = predictions.sort_values(ascending=False) #сортировка предсказанных скважин
    top_target = target.loc[top_pred.index].head(200) #200 реальных скважин
    #top_target = top_target.reset_index(drop=True)
    volume_target = top_target.sum()
    income = volume_target*point_income - investments
    return income #возвращаем разность прибыли с реальной скважины и всех затрат

Посчитаем риски убытков и прибыль для каждого региона с помощью применения техники Bootstrap:

In [77]:
#применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли
#Регион 1
state = np.random.RandomState(12345)

target_valid_1=target_valid_1.reset_index(drop=True)
 
values_1 = []

for i in range(1000):
    target_subsample_1 = target_valid_1.sample(n=points, replace=True, random_state=state)
    probs_subsample_1 = pd.Series(predicted_valid_1)[target_subsample_1.index] 
    values_1.append(revenue(target_subsample_1, probs_subsample_1))
    
values_1 = pd.Series(values_1)
print("Риск убытков", values_1[values_1<0].count()/len(values_1)*100,'%')
lower_1 = values_1.quantile(0.025) 
upper_1 = values_1.quantile(0.975) 

mean_1 = values_1.mean()
print("Средняя выручка:", mean_1)
print("Доверительный интервал", lower_1, upper_1)


Риск убытков 6.0 %
Средняя выручка: 425938526.91059244
Доверительный интервал -102090094.83793654 947976353.3583689


In [78]:
#Регион 2
state = np.random.RandomState(12345)

target_valid_2=target_valid_2.reset_index(drop=True)

    
values_2 = []
for i in range(1000):
    target_subsample_2 = target_valid_2.sample(n=points, replace=True, random_state=state)
    probs_subsample_2 = pd.Series(predicted_valid_2)[target_subsample_2.index] 
    values_2.append(revenue(target_subsample_2, probs_subsample_2))
    

values_2 = pd.Series(values_2)
print("Риск убытков", values_2[values_2<0].count()/len(values_2)*100,'%')
lower_2 = values_2.quantile(0.025) 
upper_2 = values_2.quantile(0.975) 

mean_2 = values_2.mean()
print("Средняя выручка:", mean_2)
print("Доверительный интервал", lower_2, upper_2)


Риск убытков 1.0 %
Средняя выручка: 515222773.4432899
Доверительный интервал 68873225.37050176 931547591.2570494


In [79]:
#Регион 3
state = np.random.RandomState(12345)

target_valid_3=target_valid_3.reset_index(drop=True)

    
values_3 = []
for i in range(1000):
    target_subsample_3 = target_valid_3.sample(n=points, replace=True, random_state=state)
    probs_subsample_3 = pd.Series(predicted_valid_3)[target_subsample_3.index] 
    values_3.append(revenue(target_subsample_3, probs_subsample_3))
    

values_3 = pd.Series(values_3)
print("Риск убытков", values_3[values_3<0].count()/len(values_3)*100,'%')
lower_3 = values_3.quantile(0.025) 
upper_3 = values_3.quantile(0.975) 


mean_3 = values_3.mean()
print("Средняя выручка:", mean_3)
print("Доверительный интервал", lower_3, upper_3)


Риск убытков 6.4 %
Средняя выручка: 435008362.7827556
Доверительный интервал -128880547.32978901 969706954.1802661


На основе полученных данных после процедуры Bootstrap с 1000 повторений, риск убытков более 2,5% прогнозируется в регионах №1 и №3, поэтому данные регионы исключаются. У оставшегося региона 2 риск убытков 1%, среденее значение прибыли 515 млн.руб.

## ВЫВОД:
В ходе исследования выбора локации для скважины:
- Были изучены и подготовлены данные для дальнейшей работы с ними
- Созданы модели линейной регресии предсказывающие объем запаса сырья, для каждой модели запас оказался ниже среднего уровня безубыточности разработки новой скважины. Самый близкие предсказаные объема сырья к уровню безубыточному показали значения третьего региона
- Выбраны 200 лучших скважин для каждого из регионов и с помощью техники Bootstrap расчитаны средние значения прибыли для каждого региона и риски убыточности разработки месторождения. На данном этапе исследования самым оптимальным выбран регион 2 с показателем средней прибыли в 515 млн.руб. и рисками убытков в 1%, с вероятностью в 95% прибыли не ниже 69 млн.руб.